# Pospischil sPYr Example

### Install dependencies for ACT and setup environment

In [1]:
!pip install act-neuron

import os
import sys

from act import simulator, analysis
from IPython.display import Image
import numpy as np

# Download the repository with all relevant model files if in google colab
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    !rm -rf ./ACT
    !git clone https://github.com/V-Marco/ACT
    base_dir = "./ACT"
else:
    base_dir = "../.."

### Define a config file

These settings are modifiable by the user of ACT. Tune your own cells by altering values here.

In [2]:
config = {
    "cell": {
        "hoc_file": os.path.join(base_dir, "data/Pospischil/sPyr/template.hoc"),
        "modfiles_folder": os.path.join(base_dir, "data/Pospischil/sPyr/seg_modfiles"),
        "name": "sPYr",
    },
    "simulation_parameters": {
        "h_v_init": -60.0,  # (mV)
        "h_tstop": 2000,  # (ms)
        "h_i_delay": 500,  # (ms)
        "h_i_dur": 1500,  # (ms)
        "h_dt": 0.025,
    },
    "optimization_parameters": {
        "amps": list(np.arange(-2, 10, 0.1)),
        "params": [
            {"channel": "g_pas", "low": 5.0e-06, "high": 2.0e-05},
            {"channel": "gnabar_hh2", "low": 2.5e-02, "high": 1.0e-01},
            {"channel": "gkbar_hh2", "low": 2.5e-03, "high": 1.0e-02},
            {"channel": "gkbar_im", "low": 1.5e-05, "high": 6.0e-05},
            {"channel": "gcabar_it", "low": 5.0e-04, "high": 2.0e-03},
        ],
        "target_V": None,  # Target voltages
        "target_params": [1e-5, 0.05, 0.005, 3e-5, 0.001],
        "num_repeats": 3,
        "num_amps_to_match": 12,
        "num_epochs": 5000,
    },
    "summary_features": {
        "spike_threshold": 20,  # (mV)
        # Target-sim match conditions (max abs diff between sim and target)
        "mc_num_spikes": 1,
        "mc_interspike_time": 200,  # (ms)
        "mc_min_v": 1,  # (mV)
        "mc_mean_v": 2,  # (mV)
        "mc_max_v": 1,  # (mV)
    },
    "segregation": [
        {
            "params": ["g_pas"],
            "voltage": [-100, -65],
            "time": [0, 500],
        },
        {
            "params": ["gnabar_hh2", "gkbar_hh2", "gkbar_im", "gcabar_it"],
            "voltage": [-65, 100],
            "time": [0, 2000],
        },
    ],
    "output": {"folder": "output_Pospischil_sPYr", "produce_plots": True},
    "run_mode": "segregated",  # "original", "segregated"
}

### Run the simulation

In [3]:
simulator.run(config)

/home/tyler/neuro-venv/bin/nrnivmodl:10: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import working_set


INFO : Using neuron-nightly Package (Developer Version)
/home/tyler/git_stage/ACT/examples/Pospischil_sPYr
Mod files: "../../data/Pospischil/sPyr/seg_modfiles/../../data/Pospischil/sPyr/seg_modfiles/cadecay_destexhe.mod" "../../data/Pospischil/sPyr/seg_modfiles/../../data/Pospischil/sPyr/seg_modfiles/HH_traub.mod" "../../data/Pospischil/sPyr/seg_modfiles/../../data/Pospischil/sPyr/seg_modfiles/IM_cortex.mod" "../../data/Pospischil/sPyr/seg_modfiles/../../data/Pospischil/sPyr/seg_modfiles/IT_huguenard.mod"

 -> Compiling mod_func.cpp
 => LINKING shared library ./libnrnmech.so
Successfully created x86_64/special


NEURON: The user defined name already exists: cad
 near line 0
 objref hoc_obj_[2]
                   ^
        nrn_load_dll("./x86_64/....")



(2023-09-03 17:42:42.019499)-[START]
(2023-09-03 17:42:42.019582)-[INFO]: Number of amplitudes: 120
(2023-09-03 17:42:42.043671)-[INFO]: Mod files already loaded. Continuing.
Skipping analytical setting of passive properties, no cell passive_properties specified in config.
(2023-09-03 17:43:20.906994)-[INFO]: Target voltage shape: torch.Size([120, 80000])
Skipping analytical setting of passive properties, no cell passive_properties specified in config.
(2023-09-03 17:43:20.958211)-[INFO]: Matching 12 amplitudes.
(2023-09-03 17:43:59.811930)-[INFO]: Total amplitudes matched: 8/12.
(2023-09-03 17:44:36.558453)-[INFO]: Total amplitudes matched: 13/12.
(2023-09-03 17:44:36.562531)-[INFO]: Matched amplitudes: [0.7 2.  2.9 3.1 4.1 4.6 4.7 5.  5.2 6.  6.2 8.6 9.1]
(2023-09-03 17:44:36.805453)-[INFO]: Training a model with SGD optimizer and lr = 1e-8.
(2023-09-03 17:44:36.805536)-[INFO]: Number of trainable parameters: 20546305
(2023-09-03 17:44:36.809518)-[EPOCH 0]: l1_loss = 0.0
(2023-09-03

In [ ]:
Image(filename="./output_Pospischil_sPYr/segregated/1000nA.png")

In [ ]:
analysis.print_run_stats(config)

### Compare with the non-segregated approach

In [ ]:
# Update our config to use the original mod files
config["cell"]["modfiles_folder"] = os.path.join(
    base_dir, "data/Pospischil/sPyr/orig_modfiles"
)

# Change the run mode to original (instead of segregated)
config["run_mode"] = "original"

# Re-run

simulator.run(config)

In [ ]:
Image(filename="./output_Pospischil_sPYr/original/1000nA.png")

In [ ]:
analysis.print_run_stats(config)

### Analytical setting of passive properties

In this section, let's set the passive properties for the cell so they can be analytically determined instead of using a model network.

In [ ]:
config = {
    "cell": {
        "hoc_file": os.path.join(base_dir, "data/Pospischil/sPyr/template.hoc"),
        "modfiles_folder": os.path.join(base_dir, "data/Pospischil/sPyr/seg_modfiles"),
        "name": "sPYr",
        "passive_properties": {
            "v_rest": -60,
            "r_in": 393.45,
            "tau": 117.425,
            "leak_conductance_variable": "g_pas",  # eg: g_leak
            "leak_reversal_variable": "e_pas",  # eg: e_leak
        },
    },
    "simulation_parameters": {
        "h_v_init": -60.0,  # (mV)
        "h_tstop": 2000,  # (ms)
        "h_i_delay": 500,  # (ms)
        "h_i_dur": 1500,  # (ms)
        "h_dt": 0.025,
    },
    "optimization_parameters": {
        "amps": list(np.arange(-2, 10, 0.1)),
        "params": [
            {"channel": "gnabar_hh2", "low": 2.5e-02, "high": 1.0e-01},
            {"channel": "gkbar_hh2", "low": 2.5e-03, "high": 1.0e-02},
            {"channel": "gkbar_im", "low": 1.5e-05, "high": 6.0e-05},
            {"channel": "gcabar_it", "low": 5.0e-04, "high": 2.0e-03},
        ],
        "target_V": None,  # Target voltages
        "target_params": [0.05, 0.005, 3e-5, 0.001],
        "num_repeats": 3,
        "num_amps_to_match": 12,
        "num_epochs": 5000,
    },
    "summary_features": {
        "spike_threshold": 20,  # (mV)
        # Target-sim match conditions (max abs diff between sim and target)
        "mc_num_spikes": 1,
        "mc_interspike_time": 200,  # (ms)
        "mc_min_v": 1,  # (mV)
        "mc_mean_v": 2,  # (mV)
        "mc_max_v": 1,  # (mV)
    },
    "segregation": [
        {
            "params": ["gnabar_hh2", "gkbar_hh2", "gkbar_im", "gcabar_it"],
            "voltage": [-65, 100],
            "time": [0, 2000],
        },
    ],
    "output": {"folder": "output_Pospischil_sPYr_p", "produce_plots": True},
    "run_mode": "segregated",  # "original", "segregated"
}

In [ ]:
simulator.run(config)

In [ ]:
Image(filename="./output_Pospischil_sPYr_p/segregated/1000nA.png")

In [ ]:
analysis.print_run_stats(config)